<a href="https://colab.research.google.com/github/SATYA1962S/digit_1/blob/main/backpack_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install kaggle

In [61]:
from google.colab import files
files.upload()  # Upload the kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"satya1962s","key":"cced216dc9a3815ad633d14956591283"}'}

In [62]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [63]:
!mkdir -p ~/.config/kaggle  # Create the correct directory
!mv /content/kaggle.json ~/.config/kaggle/  # Move the file to the new directory
!chmod 600 ~/.config/kaggle/kaggle.json  # Set permissions

In [64]:
!kaggle competitions download -c playground-series-s5e2

playground-series-s5e2.zip: Skipping, found more recently modified local copy (use --force to force download)


In [65]:
!unzip playground-series-s5e2.zip


Archive:  playground-series-s5e2.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [66]:
df_train = pd.read_csv('train.csv')
df_train_extra = pd.read_csv('training_extra.csv')
df_test = pd.read_csv('test.csv')

In [67]:
df_train_extra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3694318 entries, 0 to 3694317
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Brand                 object 
 2   Material              object 
 3   Size                  object 
 4   Compartments          float64
 5   Laptop Compartment    object 
 6   Waterproof            object 
 7   Style                 object 
 8   Color                 object 
 9   Weight Capacity (kg)  float64
 10  Price                 float64
dtypes: float64(3), int64(1), object(7)
memory usage: 310.0+ MB


In [68]:
df_train_len = len(df_train)
df_test_len = len(df_test)
df_train_extra_len = len(df_train_extra)
print(df_train_len,df_test_len,df_train_extra_len)

300000 200000 3694318


In [69]:
def df_combine1(df_train,df_train_extra):
    df_combined = pd.concat([df_train,df_train_extra], ignore_index= True)
    return df_combined


full_df = df_combine1(df_train,df_train_extra)
full_df_len =len(full_df)

In [70]:
full_df.isna().sum()

,0
id,0
Brand,126758
Material,110962
Size,87785
Compartments,0
Laptop Compartment,98533
Waterproof,94324
Style,104180
Color,133617
Weight Capacity (kg),1808


In [71]:
full_df.rename({'Laptop Compartment':'Laptopc','Weight Capacity (kg)':'Weght'},axis =1, inplace = True)

In [72]:
print(full_df)

              id         Brand   Material    Size  Compartments Laptopc  \
0              0      Jansport    Leather  Medium           7.0     Yes   
1              1      Jansport     Canvas   Small          10.0     Yes   
2              2  Under Armour    Leather   Small           2.0     Yes   
3              3          Nike      Nylon   Small           8.0     Yes   
4              4        Adidas     Canvas  Medium           1.0     Yes   
...          ...           ...        ...     ...           ...     ...   
3994313  4194313          Nike     Canvas     NaN           3.0     Yes   
3994314  4194314          Puma    Leather   Small          10.0     Yes   
3994315  4194315      Jansport     Canvas   Large          10.0      No   
3994316  4194316          Puma     Canvas     NaN           2.0      No   
3994317  4194317  Under Armour  Polyester  Medium           2.0     Yes   

        Waterproof      Style  Color      Weght      Price  
0               No       Tote  Black  

In [73]:
print('Categorical Variables')
Categorical_variables = full_df.select_dtypes(include=['object']).columns
for col in full_df.select_dtypes(include=['object']).columns:
    print(col)

Categorical Variables
Brand
Material
Size
Laptopc
Waterproof
Style
Color


In [74]:

print('Numerical variables')
Numerical_variables = full_df.select_dtypes(include=['int64', 'float64']).columns
for col in full_df.select_dtypes(include=['int64', 'float64']).columns:
    print(col)

Numerical variables
id
Compartments
Weght
Price


In [75]:
for cat_col in Categorical_variables:
    full_df[cat_col] = full_df[cat_col].fillna(full_df[cat_col].mode()[0])

In [76]:
for num_col in Numerical_variables:
    full_df[num_col] = full_df[num_col].fillna(full_df[num_col].median())

In [77]:
y = full_df['Price']
X = full_df.drop('Price', axis=1)  # Features
               # Target variable

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [78]:
X.isna().sum()

,0
id,0
Brand,0
Material,0
Size,0
Compartments,0
Laptopc,0
Waterproof,0
Style,0
Color,0
Weght,0


In [79]:
y = y.astype(int)  # Convert float labels to integers

In [80]:
print(y.dtype)        # Should output `int64` or `object` (for strings)
print(y.unique())     # Should show discrete classes (e.g., [0, 1, 2])

int64
[112  68  39  80  86  20  84  27  25  38  23 111  26  75  37  69  59  35
 132 106  90  62  96  49 107 138  40  56  22 128  42  53  58  72  78  36
 102 137  98  17  52  66 116  18 134  24 149 126  92  48 109 136 122  77
  81 131 121  30 110  76  57  94  67 125 101  60 143 142 120  87 146  88
  28  34  83  16  55  41 150  79 139 133  97  45  64  21 108  99  31  51
  47 144  33 100 140  46  93  54 127 135  85 103  82  63 123 119 130 141
  70 105  50  43 129 114  29  19 118 115  95 104 147 113  73  44  74 124
  15 148  91  89  32  65  71 145 117  61]


In [81]:
Numerical_variables = full_df.select_dtypes(include=['int64', 'float64']).columns

In [82]:

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), Categorical_variables)
    ],
    remainder='passthrough'  # Keeps numerical columns unchanged
)


In [83]:
from sklearn.linear_model import LinearRegression  # Or use RandomForestRegressor, GradientBoostingRegressor, etc.

pipe = Pipeline(steps=[
    {"preprocessor", preprocessor},
    {"regressor", LinearRegression()}  # Changed to a regression model
])





In [84]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [85]:
from sklearn.linear_model import LinearRegression
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

# Convert target to integer type
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Train the model
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Brand', 'Material', 'Size', 'Laptopc', 'Waterproof', 'Style', 'Color'], dtype='object'))])),
                ('regressor', LinearRegression())])

In [86]:

pipe = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LinearRegression())
])

# Fit and predict on a small subset
pipe.fit(X.head(), y.head())  # No error now
pipe.predict(X.head())

array([112.,  68.,  39.,  80.,  86.])

In [87]:
from sklearn.metrics import mean_absolute_percentage_error

# Fit the pipeline
pipe.fit(X_train, y_train)

# Predict and compute MAPE
y_pred = pipe.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # Convert to percentage
print(f"MAPE: {mape:.2f}%")

MAPE: 69.15%


In [88]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
print(f"MAPE: {mape:.2f}%")

MAPE: 69.15%


In [89]:
!pip install xgboost

In [90]:
from xgboost import XGBRegressor
pipe = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor())  # Gradient-boosted trees
])

In [91]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "regressor__n_estimators": [100, 200],
    "regressor__max_depth": [3, 5]
}
grid_search = GridSearchCV(pipe, param_grid, cv=5)
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         Index(['Brand', 'Material', 'Size', 'Laptopc', 'Waterproof', 'Style', 'Color'], dtype='object'))])),
                                       ('regressor',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     colsample_bylevel=None,
                                                     co...
                                                     learning_rate=None,
                                                     max_bin=None,
                                                     max_cat_threshold=None,
                                                     max_cat_to_onehot=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             param_grid={'regressor__max_depth': [3, 5],
                         'regressor__n_estimators': [100, 200]})

In [92]:
y_train_transformed = np.log1p(y_train)
pipe.fit(X_train, y_train_transformed)
y_pred = np.expm1(pipe.predict(X_test))  # Revert after prediction

In [93]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)

In [94]:
print(mae)

34.50055694580078


In [95]:
df_test.isna().sum()

,0
id,0
Brand,6227
Material,5613
Size,4381
Compartments,0
Laptop Compartment,4962
Waterproof,4811
Style,5153
Color,6785
Weight Capacity (kg),77


In [96]:
df_test.rename({'Laptop Compartment':'Laptopc','Weight Capacity (kg)':'Weght'},axis =1, inplace = True)

In [97]:
print('Categorical Variables')
Categorical_variables = df_test.select_dtypes(include=['object']).columns
for col in df_test.select_dtypes(include=['object']).columns:
    print(col)

Categorical Variables
Brand
Material
Size
Laptopc
Waterproof
Style
Color


In [98]:

print('Numerical variables')
Numerical_variables = df_test.select_dtypes(include=['int64', 'float64']).columns
for col in df_test.select_dtypes(include=['int64', 'float64']).columns:
    print(col)

Numerical variables
id
Compartments
Weght


In [99]:
for cat_col in Categorical_variables:
    df_test[cat_col] = df_test[cat_col].fillna(df_test[cat_col].mode()[0])

In [100]:
for num_col in Numerical_variables:
    df_test[num_col] = df_test[num_col].fillna(df_test[num_col].median())

In [101]:
df_test.isna().sum()

,0
id,0
Brand,0
Material,0
Size,0
Compartments,0
Laptopc,0
Waterproof,0
Style,0
Color,0
Weght,0


In [102]:
y_pred = grid_search.predict(df_test)

In [103]:
print(y_pred)

[80.5311   81.913795 82.647804 ... 81.762276 80.79919  80.95657 ]


In [105]:
output = pd.DataFrame({'id':df_test['id'], 'Price': y_pred})
output.to_csv('submission.csv',index = False)
print("Your submission has been successfully submitted.")

Your submission has been successfully submitted.


In [106]:
!kaggle competitions submit -c playground-series-s5e2 -f submission.csv -m "Message"

100% 3.08M/3.08M [00:01<00:00, 2.75MB/s]
Successfully submitted to Backpack Prediction Challenge